<a href="https://colab.research.google.com/github/mittushaji25/crypto-xrp-analysis/blob/main/notebooks/01_xrpusdt_data_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📈 XRPUSDT Crypto Market ETL Pipeline: Extraction, Transformation & Loading

This notebook demonstrates a complete ETL (Extract, Transform, Load) pipeline for the XRPUSDT trading pair using Binance API data. It covers:

- Extraction of raw market data including price, volume, and timestamps

- Transformation into a clean, structured format suitable for analysis

- Loading into a DataFrame for further exploration, visualization, or model training

The workflow sets the foundation for deeper crypto analytics, such as trend detection, volatility tracking, and predictive modeling.

## Step 1: Data Extraction

This step extracts historical XRPUSDT trading data from Binance using their public API. The data includes timestamped information and is saved as a raw CSV file for further cleaning and analysis.

### Key Tasks:
Connect to Binance API
Fetch monthly data starting from January 2023 to till date
Unzip individual montly files and combine data to one for reproducibility

In [1]:
import requests, zipfile, io
import pandas as pd
from datetime import datetime

In [2]:
years = [2023, 2024, 2025]
months = [f"{i:02d}" for i in range(1, 13)]

all_data = pd.DataFrame()

for year in years:
    for month in months:
        # Skip future months in 2025
        if year == 2025 and int(month) > datetime.now().month:
            continue

        url = f"https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1d/XRPUSDT-1d-{year}-{month}.zip"
        try:
            response = requests.get(url)
            response.raise_for_status()

            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                z.extractall("xrpusdt_data")
                csv_name = z.namelist()[0]

            df = pd.read_csv(f"xrpusdt_data/{csv_name}", header=None)
            df.columns = [
                "Open Time", "Open", "High", "Low", "Close", "Volume",
                "Close Time", "Quote Asset Volume", "Number of Trades",
                "Taker Buy Base Volume", "Taker Buy Quote Volume", "Ignore"
            ]
            df['Source Filename'] = csv_name

            all_data = pd.concat([all_data, df], ignore_index=True)
        except Exception as e:
            print(f"❌ Failed for {year}-{month}: {e}")

❌ Failed for 2025-07: 404 Client Error: Not Found for url: https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1d/XRPUSDT-1d-2025-07.zip


In [3]:
all_data.to_csv("xrpusdt_combined.csv", index=False)

# Step 2: Data Cleaning & Transformation

Explore the raw XRPUSDT dataset to understand its structure, identify potential issues, and guide the next steps in cleaning and transformation.

**Key Questions:**

- What columns are present, and what do they represent?
- Are there missing values or duplicates?
- Are timestamps valid and consistent?
- Are there outliers or anomalies in price or volume?
- What is the overall shape and distribution of the data?

In [4]:
all_data.shape
all_data.columns
all_data.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,Ignore,Source Filename
0,1672531200000,0.3389,0.3400,0.3354,0.3387,102026821.0,1672617599999,3.447164e+07,48661,53592277.0,1.810790e+07,0,XRPUSDT-1d-2023-01.csv
1,1672617600000,0.3386,0.3555,0.3000,0.3482,647520487.0,1672703999999,2.178040e+08,285660,320823575.0,1.080569e+08,0,XRPUSDT-1d-2023-01.csv
2,1672704000000,0.3482,0.3520,0.3395,0.3436,346410334.0,1672790399999,1.195113e+08,154113,172318472.0,5.944846e+07,0,XRPUSDT-1d-2023-01.csv
3,1672790400000,0.3436,0.3605,0.3390,0.3472,372672796.0,1672876799999,1.295853e+08,147677,193027855.0,6.713928e+07,0,XRPUSDT-1d-2023-01.csv
4,1672876800000,0.3471,0.3487,0.3333,0.3380,265529668.0,1672963199999,9.073585e+07,106910,129294274.0,4.418781e+07,0,XRPUSDT-1d-2023-01.csv


In [5]:
all_data.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,Ignore,Source Filename
0,1672531200000,0.3389,0.3400,0.3354,0.3387,102026821.0,1672617599999,3.447164e+07,48661,53592277.0,1.810790e+07,0,XRPUSDT-1d-2023-01.csv
1,1672617600000,0.3386,0.3555,0.3000,0.3482,647520487.0,1672703999999,2.178040e+08,285660,320823575.0,1.080569e+08,0,XRPUSDT-1d-2023-01.csv
2,1672704000000,0.3482,0.3520,0.3395,0.3436,346410334.0,1672790399999,1.195113e+08,154113,172318472.0,5.944846e+07,0,XRPUSDT-1d-2023-01.csv
3,1672790400000,0.3436,0.3605,0.3390,0.3472,372672796.0,1672876799999,1.295853e+08,147677,193027855.0,6.713928e+07,0,XRPUSDT-1d-2023-01.csv
4,1672876800000,0.3471,0.3487,0.3333,0.3380,265529668.0,1672963199999,9.073585e+07,106910,129294274.0,4.418781e+07,0,XRPUSDT-1d-2023-01.csv


In [6]:
all_data.tail()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,Ignore,Source Filename
907,1750896000000000,2.1858,2.2127,2.0956,2.1054,120378866.1,1750982399999999,2.588798e+08,871990,62523800.2,1.346067e+08,0,XRPUSDT-1d-2025-06.csv
908,1750982400000000,2.1053,2.1556,2.0671,2.1419,120577000.3,1751068799999999,2.533541e+08,917776,63983419.8,1.344556e+08,0,XRPUSDT-1d-2025-06.csv
909,1751068800000000,2.1420,2.2079,2.1336,2.1854,70042779.0,1751155199999999,1.530003e+08,648620,34020502.0,7.430289e+07,0,XRPUSDT-1d-2025-06.csv
910,1751155200000000,2.1855,2.2198,2.1754,2.2061,46097897.5,1751241599999999,1.011689e+08,381555,21341560.4,4.685453e+07,0,XRPUSDT-1d-2025-06.csv
911,1751241600000000,2.2062,2.3271,2.1650,2.2362,151525906.2,1751327999999999,3.391740e+08,1132152,77672364.0,1.738165e+08,0,XRPUSDT-1d-2025-06.csv


In [7]:
# Back-up of original data
all_data_copy = all_data.copy()

### Timestamp Conversion Issue & Resolution

While attempting to convert the `Open Time` and `Close Time` columns to datetime format, we encountered an error:

`OutOfBoundsDatetime: Out of bounds nanosecond timestamp`

Investigation:
**bold text**
- The issue was traced back to data from 2025, while data from 2023 and 2024 converted correctly.

- To isolate the problem, modified the data extraction process to include a new column: `Source Filename`, which stores the original filename (e.g., XRPUSDT-1d-2025-01.csv).

- On further exploration identified, timestamps in 2025 files are in microseconds (us), whereas 2023 and 2024 files use milliseconds (ms).
Resolution Strategy:

- Use the `Source Filename` column to determine the year of each row.

Apply timestamp conversion based on the year:
- `unit='ms'` for 2023 and 2024
- `unit='us'` for 2025
- Use `errors='coerce'` to safely handle any invalid timestamps by converting them to `NaT`.

In [8]:
def fix_timestamp(ts):
    try:
        ts = int(ts)
        if ts > 1e14:  # microseconds
            return pd.to_datetime(ts, unit='us', errors='coerce')
        elif ts > 1e11:  # milliseconds
            return pd.to_datetime(ts, unit='ms', errors='coerce')
        else:  # seconds
            return pd.to_datetime(ts, unit='s', errors='coerce')
    except:
        return pd.NaT

# Apply to original timestamp column
all_data['Open Time'] = all_data['Open Time'].apply(fix_timestamp)
all_data['Close Time'] = all_data['Close Time'].apply(fix_timestamp)

In [9]:
# Check data types
all_data.dtypes

,0
Open Time,datetime64[ns]
Open,float64
High,float64
Low,float64
Close,float64
Volume,float64
Close Time,datetime64[ns]
Quote Asset Volume,float64
Number of Trades,int64
Taker Buy Base Volume,float64


In [10]:
# Summary statistics
all_data.describe()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,Ignore
count,912,912.000000,912.000000,912.000000,912.000000,9.120000e+02,912,9.120000e+02,9.120000e+02,9.120000e+02,9.120000e+02,912.0
mean,2024-03-31 12:00:00,0.970641,1.003148,0.936285,0.972723,3.700959e+08,2024-04-01 11:59:59.999198720,3.492578e+08,7.728642e+05,1.833062e+08,1.726876e+08,0.0
min,2023-01-01 00:00:00,0.338000,0.340000,0.300000,0.338000,4.052670e+07,2023-01-01 23:59:59.999000,2.750721e+07,3.836700e+04,1.998583e+07,1.261434e+07,0.0
25%,2023-08-16 18:00:00,0.502775,0.511175,0.488975,0.502950,1.887051e+08,2023-08-17 17:59:59.999000064,1.304329e+08,1.558885e+05,9.157859e+07,6.367682e+07,0.0
50%,2024-03-31 12:00:00,0.561300,0.576600,0.543200,0.562050,3.031664e+08,2024-04-01 11:59:59.999000064,1.949285e+08,2.870230e+05,1.521314e+08,9.759324e+07,0.0
75%,2024-11-14 06:00:00,0.816275,0.930425,0.770550,0.837975,4.425727e+08,2024-11-15 05:59:59.999000064,3.478961e+08,8.052872e+05,2.201800e+08,1.704561e+08,0.0
max,2025-06-30 00:00:00,3.292300,3.400000,3.175800,3.292200,2.543717e+09,2025-06-30 23:59:59.999999,6.398461e+09,1.149188e+07,1.284592e+09,3.232098e+09,0.0
std,NaN,0.792662,0.824639,0.757598,0.793491,2.941272e+08,NaN,5.077632e+08,1.211049e+06,1.468311e+08,2.526205e+08,0.0


In [11]:
all_data.drop(columns=['Ignore'], inplace=True)


In [12]:
# Standardize column name for consistency and easy access
all_data.columns = [col.strip().replace(" ", "_").lower() for col in all_data.columns]

In [13]:
all_data.to_csv("xrpusdt_data_cleaned.csv", index=False)